# Vaex Run

In [6]:
import os
import vaex as vx
import time
import numpy as np
import shutil
import gc
import joblib
import psutil

In [7]:
# PATH_DATA = '../../df-showdown-data/'
PATH_DATA = 'O:/df-showdown-data/'
TEMP_PATH = PATH_DATA+'temp/'
PATH_OUTPUT = '../output/'

SLA_NAME = 'pandas-sla.pkl'
MEM_NAME = 'pandas-mem.pkl'


In [8]:
all_sla = {}
all_memory = {}

In [9]:
def get_memory():
    est_memory = psutil.virtual_memory()[3]/1000000000
    return est_memory

##     Task1 & Task2- I/O

In [10]:
all_sla['task1'] = {}
all_sla['task1']['csv'] = {}
all_sla['task1']['prq'] = {}

all_sla['task2'] = {}
all_sla['task2']['csv'] = {}
all_sla['task2']['prq'] = {}



all_memory['task1'] = {}
all_memory['task1']['csv'] = {}
all_memory['task1']['prq'] = {}

all_memory['task2'] = {}
all_memory['task2']['csv'] = {}
all_memory['task2']['prq'] = {}


In [11]:
for i in range(10):
# for i in [6,]:
    print(i)
    fn = 'data_{}.csv'.format(i)
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1 = vx.open(PATH_DATA+fn)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['csv'][nrows]=end
    all_memory['task1']['csv'][nrows]=end_mem

    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1.export(TEMP_PATH + 'temp.csv',index=False)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    all_sla['task2']['csv'][nrows]=end
    all_memory['task2']['csv'][nrows]=end_mem

    del dtemp1
    gc.collect()
    os.remove(TEMP_PATH + 'temp.csv')
    time.sleep(2)

    fn = 'data_{}.parquet'.format(i)
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1 = vx.open(PATH_DATA+fn)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['prq'][nrows]=end
    all_memory['task1']['prq'][nrows]=end_mem
    

    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1.export(TEMP_PATH + 'temp.parquet',index=False)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    all_sla['task2']['prq'][nrows]=end
    all_memory['task2']['prq'][nrows]=end_mem

    del dtemp1
    gc.collect()
    os.remove(TEMP_PATH + 'temp.parquet')

0
1
2
3
4
5
6


ValueError: Error reading csv file O:/df-showdown-data/data_6.csv, write offending chunk to: C:\Users\amoza\AppData\Local\Temp\tmpg7csym6j.csv (len=11527509, first=False, columns=['Order ID'], schema=Order ID: string, encoding=utf8, schema_infer_fraction=0.001).
Possible causes:
  * This could be a file encoding error. Consider passing read_options=pyarrow.csv.ReadOptions(encoding="ISO-8859-1") or another encoding as argument.
  * We might have inferred the wrong schema:
     * Consider giving a schema hint by e.g. passing read_options=pyarrow.csv.ConvertOptions(column_types={"SomeId": pyarrow.string()}).
     * Consider increasing schema_infer_fraction (e.g. schema_infer_fraction=1 to parse the whole file to infer the schema).


In [ ]:
all_sla

{'task1': {'csv': {'0': 0.1597957999999995,
   '1': 0.9964031999999996,
   '5': 5.668758500000003,
   '10': 10.586043699999998,
   '25': 27.4135012,
   '50': 53.45036569999999},
  'prq': {'0': 0.09723499999999952,
   '1': 0.3995843000000008,
   '5': 1.5912186999999989,
   '10': 2.819780399999999,
   '25': 6.973793099999995,
   '50': 15.728042500000015}},
 'task2': {'csv': {'0': 0.5235167999999994,
   '1': 3.5646044999999997,
   '5': 17.534583899999998,
   '10': 34.267026,
   '25': 85.98262679999999,
   '50': 172.4047499},
  'prq': {'0': 0.10737889999999872,
   '1': 0.6508976000000004,
   '5': 2.8340707999999992,
   '10': 5.8789224000000075,
   '25': 14.509268200000008,
   '50': 28.910167699999988}}}

In [ ]:
all_memory

{'task1': {'csv': {'0': 0.013037567999999666,
   '1': 0.10687283200000053,
   '5': 0.5295185920000005,
   '10': 1.1669381120000004,
   '25': 2.9038960639999996,
   '50': 5.78781184},
  'prq': {'0': 0.06413107200000034,
   '1': 0.16606412800000037,
   '5': 0.5594767360000006,
   '10': 0.7485480960000004,
   '25': 1.7997946880000004,
   '50': 3.669024768}},
 'task2': {'csv': {'0': -0.010838015999999229,
   '1': -0.016322560000000763,
   '5': -0.035409920000001094,
   '10': -0.010731520000000216,
   '25': -0.01606451199999981,
   '50': -0.05526323199999972},
  'prq': {'0': 0.028311551999999907,
   '1': 0.08388198399999958,
   '5': 0.10481254399999962,
   '10': 0.09911091200000044,
   '25': 0.11252531200000071,
   '50': -0.06293504000000105}}}

## Get Data for other task

In [ ]:
all_df = [vx.open(PATH_DATA+'data_{}.parquet'.format(i)) for i in [4,5,6,7]]
df_right = vx.open(PATH_DATA+'data_to_join.parquet')

## Task3 - Sorting

In [ ]:
n_task = 'task3'
all_sla[n_task] = {}

In [ ]:
for dtemp1 in all_df:
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.sort_values(['Date','Amount'],ascending=[True,False])
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    gc.collect()
    time.sleep(1)


In [ ]:
all_sla[n_task]

{'3': 0.63470099999995,
 '9': 2.022421900000154,
 '19': 4.136806799999931,
 '34': 10.888667199999873,
 '45': 16.219327799999974}

In [ ]:
all_memory[n_task]

## Task4 - Filtering

In [ ]:
n_task = 'task4'
all_sla[n_task] = {}

In [ ]:
# dtemp1 = all_df[0]
filt1 = ['Shipped - Delivered to Buyer', 'Cancelled']
for dtemp1 in all_df:
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1[(dtemp1['Amount']>300) & (~dtemp1['Status'].isin(filt1))]
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    gc.collect()
    time.sleep(1)


In [ ]:
all_sla[n_task]

{'3': 0.21421140000006744,
 '9': 0.5953160999999909,
 '19': 1.2111443000001145,
 '34': 2.2449423999999,
 '45': 2.9474488999999267}

In [ ]:
all_memory[n_task]

## Task5 - Merging

In [ ]:
n_task = 'task5'
all_sla[n_task] = {}

In [ ]:
for dtemp1 in all_df:
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = pd.merge(dtemp1,df_right,on=['Date','ship-service-level'],how='left')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))

    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    gc.collect()
    time.sleep(1)

In [ ]:
all_sla[n_task]

{'3': 0.5124172000000726,
 '9': 1.4778619999997318,
 '19': 2.9784222000002956,
 '34': 5.352248900000177,
 '45': 7.045791199999712}

In [ ]:
all_memory[n_task]

## Task6 - udf apply

In [ ]:
n_task = 'task6'
all_sla[n_task] = {}

In [ ]:
def fun(x):
    x0 = x + 1
    for i in range(25):
        if x0<800:
            x0 += i
            x0 = (x0/3.0)*2.5
            x0 = x0*1.2
        else:
            x0 += i/2.0
            x0 = (x0/4.0)*3.8

    return x0

In [ ]:
# dtemp1=all_df[0]
# dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
for dtemp1 in all_df:
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem
    
    del dtemp1
    gc.collect()
    time.sleep(1)

In [ ]:
all_sla[n_task]

{'3': 15.072074999999586,
 '9': 43.90379790000043,
 '19': 86.94100830000025,
 '34': 156.61809700000003,
 '45': 202.96429179999996}

In [ ]:
all_memory[n_task]

## Task7 - aggregation

In [ ]:
n_task = 'task7'
all_sla[n_task] = {}

In [ ]:
def p25(x):
    return np.percentile(x,25)

def p75(x):
    return np.percentile(x,75)

In [ ]:
# dtemp1=all_df[0]
for dtemp1 in all_df:
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':[np.mean, np.size, p25, p75]})
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    gc.collect()
    time.sleep(1)

In [ ]:
all_sla[n_task]

{'3': 0.787891700000273,
 '9': 2.2168736999997236,
 '19': 4.148283800000172,
 '34': 7.475733299999774,
 '45': 9.774048400000083}

In [ ]:
all_memory[n_task]

## Save results

In [ ]:
joblib.dump(all_sla,PATH_OUTPUT+SLA_NAME)
joblib.dump(all_memory,PATH_OUTPUT+MEM_NAME)

['../output/pandas-sla.pkl']

In [10]:
# all_sla